# Storm surge level

Notebook environment to migrate netcdf files to zarr and geojson

In [1]:
# Optional; code formatter, installed as jupyter lab extension
#%load_ext lab_black
# Optional; code formatter, installed as jupyter notebook extension
%load_ext nb_black

<IPython.core.display.Javascript object>

### Configure OS independent paths

In [2]:
# Import standard packages
import os
import pathlib
import sys
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr

# Make root directories importable by appending root to path
cwd = pathlib.Path().resolve()
sys.path.append(os.path.dirname(cwd))

# Get root paths
home = pathlib.Path().home()
root = home.root

# Import custom functionality
from etl import p_drive, rel_root
from etl.extract import clear_zarr_information, get_geojson
from etl.keys import load_env_variables, load_google_credentials

# Define both local and remote drives
coclico_data_dir = p_drive.joinpath("11205479-coclico", "data")

C:\Users\kras\Anaconda3\envs\bathymetry_env\lib\site-packages\xarray\backends\cfgrib_.py:27: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  warnings.warn(


<IPython.core.display.Javascript object>

In [3]:
# Project paths & files (manual input)
dataset_dir = coclico_data_dir.joinpath("01_storm_surge_jrc")
dataset_historical_path = dataset_dir.joinpath("CoastAlRisk_Europe_EESSL_Historical.nc")
dataset_rcp45_path = dataset_dir.joinpath("CoastAlRisk_Europe_EESSL_RCP45.nc")
dataset_rcp85_path = dataset_dir.joinpath("CoastAlRisk_Europe_EESSL_RCP85.nc")
dataset_out_file = "CoastAlRisk_Europe_EESSL"

<IPython.core.display.Javascript object>

### load environment variables and set credentials

In [4]:
load_env_variables(env_var_keys=["MAPBOX_ACCESS_TOKEN"])
load_google_credentials(
    google_token=coclico_data_dir.joinpath("google_credentials.json")
)

Environmental variables loaded.
Google Application Credentials load into environment.


C:\SnapVolumesTemp\MountPoints\{45c63495-0000-0000-0000-100000000000}\{D2C8ADC5-14BE-4A35-B9CA-C9C2D1930AB1}\SVROOT\Users\kras\Documents\GitHub\coclicodata\etl\keys.py:33: CredentialLeakageWarning: Keys loaded from shared network drive.
  warnings.warn("Keys loaded from shared network drive.", CredentialLeakageWarning)


<IPython.core.display.Javascript object>

### write data to Zarr files (dataset dependent)

In [13]:
# open datasets
dataset_historical = xr.open_dataset(dataset_historical_path)
dataset_45rcp = xr.open_dataset(dataset_rcp45_path)
dataset_85rcp = xr.open_dataset(dataset_rcp85_path)

# check original dataset
dataset_historical

<xarray.Dataset>
Dimensions:    (row: 2242, col: 8)
Dimensions without coordinates: row, col
Data variables:
    longitude  (row) float64 ...
    latitude   (row) float64 ...
    ssl        (row, col) float64 ...
    RP         (col) float32 ...
Attributes:
    title:            European extreme storm surge level
    Institution:      Joint European Research Center, Institute of Environmen...
    Project Name:     Prototype of a first Global Integrated Coastal Impact-b...
    Project Acronym:  CoastAlRisk
    reference:        Vousdoukas MI, Voukouvalas E, Annunziato A, Giardino A,...
    email:            michail.vousdoukas@ec.europa.eu
    version:          1.0
    terms_for_use:    European Union, 1995-2015.\nReuse is authorised, provid...
    disclaimer:       Unless the following would not be permitted or valid un...

<IPython.core.display.Javascript object>

In [7]:
# rename or swap dimension names, the latter in case the name already exists as coordinate
dataset_historical = dataset_historical.rename_dims({"row": "stations"})
dataset_45rcp = dataset_45rcp.rename_dims({"row": "stations"})
dataset_85rcp = dataset_85rcp.rename_dims({"row": "stations"})
dataset_historical = dataset_historical.swap_dims({"col": "rp"})
dataset_45rcp = dataset_45rcp.swap_dims({"col": "rp"})
dataset_85rcp = dataset_85rcp.swap_dims({"col": "rp"})

# rename variables, if necessary
dataset_historical = dataset_historical.rename_vars(
    {"RP": "rp", "longitude": "lon", "latitude": "lat"}
)
dataset_45rcp = dataset_45rcp.rename_vars(
    {"RP": "rp", "longitude": "lon", "latitude": "lat"}
)
dataset_85rcp = dataset_85rcp.rename_vars(
    {"RP": "rp", "longitude": "lon", "latitude": "lat"}
)

# set some data variables to coordinates to avoid duplication of dimensions in later stage
dataset_historical = dataset_historical.set_coords(["lon", "lat", "rp"])
dataset_45rcp = dataset_45rcp.set_coords(["lon", "lat", "rp"])
dataset_85rcp = dataset_85rcp.set_coords(["lon", "lat", "rp"])

<IPython.core.display.Javascript object>

In [9]:
# concat datasets along new dimension with index values and name derived from pandas index object, if necessary
dataset = xr.concat(
    [dataset_historical, dataset_45rcp, dataset_85rcp],
    pd.Index(["Historical", "RCP45", "RCP85"], name="scenarios"),
)

<IPython.core.display.Javascript object>

In [10]:
# re-order shape of the data variables
dataset = dataset.transpose("scenarios", "stations", "rp")

<IPython.core.display.Javascript object>

In [23]:
dataset["scenarios"]

<xarray.DataArray 'scenarios' (scenarios: 3)>
array(['Historical', 'RCP45', 'RCP85'], dtype=object)
Coordinates:
  * scenarios  (scenarios) object 'Historical' 'RCP45' 'RCP85'

<IPython.core.display.Javascript object>

In [25]:
# add attributes to variables and coordinates, if necessary

# specify custom (CF convention) attributes
dataset_attributes = {"scenarios": {"long_name": "climate scenarios"}}

# add / overwrite attributes
for k, v in dataset_attributes.items():
    try:
        dataset[k].attrs = dataset_attributes[k]
    except:
        continue

<IPython.core.display.Javascript object>

In [29]:
# check the xarray dataset, best practice is to have as many as possible bold dimensions (dimension == coordinate name).
# in this way, the Front-End can access the variable directly without having to index the variable first

dataset

<xarray.Dataset>
Dimensions:    (stations: 2242, scenarios: 3, rp: 8)
Coordinates:
    lon        (stations) float64 -0.1 -0.1 -0.1 -0.1 -0.3 ... 9.9 9.9 9.9 9.9
    lat        (stations) float64 36.1 39.3 49.7 54.3 ... 57.7 58.7 64.5 64.7
  * rp         (rp) float32 5.0 10.0 20.0 50.0 100.0 200.0 500.0 1e+03
  * scenarios  (scenarios) object 'Historical' 'RCP45' 'RCP85'
Dimensions without coordinates: stations
Data variables:
    ssl        (scenarios, stations, rp) float64 1.024 1.051 ... 2.712 2.805
Attributes:
    title:            European extreme storm surge level
    Institution:      Joint European Research Center, Institute of Environmen...
    Project Name:     Prototype of a first Global Integrated Coastal Impact-b...
    Project Acronym:  CoastAlRisk
    reference:        Vousdoukas MI, Voukouvalas E, Annunziato A, Giardino A,...
    email:            michail.vousdoukas@ec.europa.eu
    version:          1.0
    terms_for_use:    European Union, 1995-2015.\nReuse is authorised, provid...
    disclaimer:       Unless the following would not be permitted or valid un...

<IPython.core.display.Javascript object>

In [15]:
# export to zarr in write mode (to overwrite if exists)
dataset.to_zarr(dataset_dir.joinpath("%s.zarr" % dataset_out_file), mode="w")

<IPython.core.display.Javascript object>